In [668]:
%run utils.ipynb

In [676]:
import pandas as pd
import numpy as np
import os
import sys
# from utils import *
import matplotlib.pyplot as plt
%matplotlib inline
import dill

In [707]:
import sklearn
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, top_k_accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, make_pipeline

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout, Activation, Input, Embedding, TextVectorization, Reshape, Add, Concatenate, Flatten
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [5]:
TRAIN = 5000
SEG_SIZE = 100
TRAIN_SEG = round(TRAIN / SEG_SIZE)
LABELED_USERS = 10
EMBEDDING_DIM = 128

In [426]:
_, seg_df = load_user_data()
print(seg_df.shape)
y_seg = pd.read_csv('challengeToFill.csv').drop(columns=['Unnamed: 0']).T.reset_index(drop=True).T
y_seg_train = y_seg.iloc[:LABELED_USERS, :TRAIN_SEG]
y_seg_test = y_seg.iloc[:LABELED_USERS, TRAIN_SEG:]
train_df = seg_df.iloc[:LABELED_USERS, :TRAIN_SEG]
test_df = seg_df.iloc[:LABELED_USERS, TRAIN_SEG:]
submission_df = seg_df.iloc[LABELED_USERS:, TRAIN_SEG:]
# sentences = train_df.to_numpy().flatten().tolist()
# embedding_model = create_embeddings(sentences, vector_size=EMBEDDING_DIM, window=5)
# print(embedding_model.wv)
print(train_df.shape)
print(test_df.shape)

(40, 150)
(10, 50)
(10, 100)


In [427]:
tom = [' '.join(x) for x in train_df.to_numpy().flatten()]

In [626]:
tfidf_vec = TfidfVectorizer(ngram_range=(1,3), min_df=0.01, max_df=0.1, sublinear_tf=True)
mich = pd.DataFrame.sparse.from_spmatrix(tfidf_vec.fit_transform(tom), columns=tfidf_vec.get_feature_names_out())
mich['user'] = pd.Series([idx//TRAIN_SEG for idx in mich.index])
mich

,20,20 csh,20 csh xwsh,4dwm cat,4dwm cat hostname,4dwm toolches xwsh,4dwm xterm,4dwm xterm ksh,acroread,acroread acroread,...,xterm rm ksh,xterm xterm,xterm xterm ksh,xwsh csh,xwsh csh gettxt,xwsh launchef,xwsh launchef sh,xwsh sh,xwsh sh sh,user
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9


In [627]:
X_train = mich.sample(frac=1)
y_train_user = X_train['user']
X_train.drop(columns=['user'], inplace=True)

In [656]:
# clf = RandomForestClassifier(random_state=42, max_depth=3, min_samples_leaf=30, min_samples_split=15, max_features='log2')
# clf = MLPClassifier(random_state=42, learning_rate='adaptive', hidden_layer_sizes=(64,))
clf = KNeighborsClassifier(n_neighbors=10, weights='distance')
clf.fit(X_train, y_train_user)

KNeighborsClassifier(n_neighbors=10, weights='distance')

In [657]:
train_preds = clf.predict(X_train)
train_probas = clf.predict_proba(X_train)

In [658]:
top_k_accuracy_score(y_train_user, train_probas, k=1)

1.0

In [698]:
test = [' '.join(x) for x in test_df.to_numpy().flatten()]
X_test = pd.DataFrame.sparse.from_spmatrix(tfidf_vec.transform(test))
y_test_user = pd.Series([idx//(2*TRAIN_SEG) for idx in test_df.index])
X_test

,0,1,2,3,4,5,6,7,8,9,...,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [699]:
test_preds = clf.predict(X_test)
test_probas = clf.predict_proba(X_test)

In [661]:
top_k_accuracy_score(y_test_user, test_probas, k=1)

0.877

In [662]:
def get_top_k_probas(probas, k=5):
    return np.partition(probas, -k, axis=1)[:, -k:]

In [663]:
def get_top_k_classes(probas, k=5):
    return np.argpartition(probas, -k, axis=1)[:, -k:]

In [672]:
y_pred_seg = np.array([1 if t in p else 0 for t, p in zip(y_test, get_top_k_classes(test_probas, k=3))])
y_pred_seg.shape

(1000,)

In [673]:
get_metrics(y_test, y_pred_seg)

Acc: 0.899
Precision: 0.4972067039106145
Recall: 0.89
F1 Score: 0.6379928315412186
Final Grade: 100


100

In [675]:
clf_name = clf.__class__.__name__
clf_name

'KNeighborsClassifier'

In [692]:
f = FunctionTransformer(lambda y: [' '.join(x) for x in y.to_numpy().flatten()])
pipe = make_pipeline(f, tfidf_vec, clf)
pipe

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function <lambda> at 0x00000210FDA50F70>)),
                ('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.1, min_df=0.01, ngram_range=(1, 3),
                                 sublinear_tf=True)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_neighbors=10, weights='distance'))])

In [701]:
pipe.fit(train_df, y_train_user)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function <lambda> at 0x00000210FDA50F70>)),
                ('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.1, min_df=0.01, ngram_range=(1, 3),
                                 sublinear_tf=True)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_neighbors=10, weights='distance'))])

In [704]:
y_train_user

393    7
252    5
398    7
153    3
42     0
      ..
146    2
32     0
169    3
492    9
444    8
Name: user, Length: 500, dtype: int64

In [706]:
pipe.predict_proba(test_df)

array([[0.11081319, 0.        , 0.        , ..., 0.        , 0.21537191,
        0.21678979],
       [0.10879529, 0.        , 0.        , ..., 0.        , 0.19865592,
        0.21558972],
       [0.10801773, 0.        , 0.        , ..., 0.        , 0.19767328,
        0.21612284],
       ...,
       [0.        , 0.        , 0.        , ..., 0.30022187, 0.09968941,
        0.10035119],
       [0.        , 0.        , 0.        , ..., 0.30029692, 0.09961168,
        0.10038326],
       [0.        , 0.        , 0.        , ..., 0.30027685, 0.09958094,
        0.10041586]])

In [703]:
with open(f'{clf_name}.dill', 'wb') as f:
    dill.dump(pipe, f)

In [723]:
X_train = seg_df.iloc[:, :TRAIN_SEG]
X_train.shape

(40, 50)

In [764]:
func = FunctionTransformer(lambda y: [' '.join(x) for x in y.to_numpy().flatten()])
count_vec = CountVectorizer(lowercase=False)
pipe = make_pipeline(func, count_vec)
tom = pd.DataFrame.sparse.from_spmatrix(pipe.fit_transform(X_train), columns=count_vec.get_feature_names_out()).sparse.to_dense() #.astype(int) #.mode(axis=1)
tom
# tom = tom.astype(np.int32)

,20,4Dwm,9term,FIFO,LOCK,Mail,MediaMai,Sqpe,UNLOCK,Xsession,...,xmodmap,xprop,xrdb,xsessio,xset,xsetroot,xterm,xupdate,xwsh,yppasswd
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,0,0,2,2,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1996,0,0,0,2,2,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1997,0,0,0,2,2,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1998,0,0,0,2,2,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [823]:
pd.DataFrame(pd.DataFrame(train_df.to_numpy().reshape(-1,1))[0].values.tolist()).mode(1, dropna=True)[0]

0       sh
1       sh
2       sh
3       ls
4       sh
      ... 
495    ksh
496    ksh
497    ksh
498    ksh
499    ksh
Name: 0, Length: 500, dtype: object

In [832]:
train_df.to_numpy().flatten().tolist() == pd.DataFrame(train_df.to_numpy().flatten())[0].values.tolist()

True

In [835]:
from scipy import stats

In [937]:
unique_transormer = FunctionTransformer(lambda y: y.applymap(lambda x: len(set(x))))

In [872]:
mode_transformer = FunctionTransformer(lambda x: stats.mode(x.to_numpy().flatten().tolist(), axis=1).mode)

In [873]:
X_train = pd.DataFrame(train_df.to_numpy().flatten(), columns=['seg'])
X_train

,seg
0,"[cat, nawk, nawk, uname, pwd, echo, echo, ksh,..."
1,"[nawk, nawk, uname, pwd, echo, echo, ksh, unam..."
2,"[nawk, uname, pwd, echo, echo, ksh, uname, stt..."
3,"[uname, pwd, echo, echo, ksh, uname, stty, tse..."
4,"[pwd, echo, echo, ksh, uname, stty, tset, ksh,..."
...,...
495,"[ls, id, ping, rlogin, rlogin, rlogin, id, rlo..."
496,"[id, ping, rlogin, rlogin, rlogin, id, rlogin,..."
497,"[ping, rlogin, rlogin, rlogin, id, rlogin, rlo..."
498,"[rlogin, rlogin, rlogin, id, rlogin, rlogin, k..."


In [874]:
ct = make_column_transformer((mode_transformer, ['seg']), remainder='passthrough')
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('functiontransformer',
                                 FunctionTransformer(func=<function <lambda> at 0x000002110B1EEB80>),
                                 ['seg'])])

In [939]:
X_train

,seg
0,"[cat, nawk, nawk, uname, pwd, echo, echo, ksh,..."
1,"[nawk, nawk, uname, pwd, echo, echo, ksh, unam..."
2,"[nawk, uname, pwd, echo, echo, ksh, uname, stt..."
3,"[uname, pwd, echo, echo, ksh, uname, stty, tse..."
4,"[pwd, echo, echo, ksh, uname, stty, tset, ksh,..."
...,...
495,"[ls, id, ping, rlogin, rlogin, rlogin, id, rlo..."
496,"[id, ping, rlogin, rlogin, rlogin, id, rlogin,..."
497,"[ping, rlogin, rlogin, rlogin, id, rlogin, rlo..."
498,"[rlogin, rlogin, rlogin, id, rlogin, rlogin, k..."


In [938]:
X_train.applymap(lambda x: len(set(x)))

,seg
0,18
1,18
2,18
3,18
4,18
...,...
495,24
496,25
497,25
498,24


In [936]:
unique_transormer.fit_transform(X_train)

1

In [929]:
X_train.iloc[0].values[0]

['cat',
 'nawk',
 'nawk',
 'uname',
 'pwd',
 'echo',
 'echo',
 'ksh',
 'uname',
 'stty',
 'tset',
 'ksh',
 'movemail',
 'movemail',
 'uname',
 'uname',
 'pwd',
 'echo',
 'echo',
 'ksh',
 'gzip',
 'cat',
 'nawk',
 'nawk',
 'uname',
 'pwd',
 'echo',
 'echo',
 'ksh',
 'uname',
 'stty',
 'tset',
 'uname',
 'uname',
 'pwd',
 'echo',
 'echo',
 'ksh',
 'ls',
 'sendmail',
 'grep',
 'find',
 'ls',
 'ls',
 'ls',
 'ls',
 'ls',
 'ls',
 'ls',
 'ls',
 'uname',
 'uname',
 'pwd',
 'echo',
 'echo',
 'ksh',
 'uname',
 'uname',
 'pwd',
 'echo',
 'echo',
 'ksh',
 'mkdir',
 'uname',
 'uname',
 'pwd',
 'echo',
 'echo',
 'ksh',
 'ln',
 'ls',
 'ksh',
 'xgvis',
 'ls',
 'mkdir',
 'sh',
 'sh',
 'ln',
 'sh',
 'xgvis',
 'sh',
 'sh',
 'xgvis',
 'ls',
 'ln',
 'sh',
 'ls',
 'sh',
 'ln',
 'sh',
 'sh',
 'sh',
 'xgvis',
 'ln',
 'sh',
 'sh',
 'sh',
 'xgvis',
 'sh',
 'sh']

In [893]:
mode_transformer.fit_transform(X_train).shape

(500, 1)

In [902]:
pd.DataFrame(unique_transormer.fit_transform(X_train))

,0
0,"[4Dwm, [, cat, [, hostname, stty, date, echo, ..."
1,"[MediaMai, sendmail, launchef, launchef, sh, h..."
2,"[MediaMai, sendmail, launchef, sh, launchef, h..."
3,"[[, cat, [, hostname, stty, date, echo, [, fin..."
4,"[[, cat, [, stty, tset, [, uname, launchef, sh..."
...,...
495,"[xsetroot, reaper, cat, mail, csh, stty, tset,..."
496,"[xsetroot, reaper, cat, mail, tcsh, hostname, ..."
497,"[xsetroot, reaper, cat, mail, tcsh, hostname, ..."
498,"[xterm, launchef, launchef, sh, netstat, netsc..."


In [834]:
pd.DataFrame(train_df.to_numpy().flatten().tolist()).mode(1)[0]

0       sh
1       sh
2       sh
3       ls
4       sh
      ... 
495    ksh
496    ksh
497    ksh
498    ksh
499    ksh
Name: 0, Length: 500, dtype: object